# Customer's error function

To the error function it is expected the following pandas.DataFrame struncture:

| StoreID  | Month | Region | NumberOfSales | _NumberOfSales |
| ---------|----------------|---------------| ---------------|
| 1000	   |3      | 4      | 16            | 16             |
| 1000	   |4      | 4      | 30            | 23             |
| 1001	   |3      | 6      | 410           | 411            |
| 1001	   |4      | 27     | 3130          | 3120           |
| 1002	   |3      | 58     | 10            | 8              |

Where:
 
 - *NumberOfSales* are the test values
 - *_NumberOfSales* are the predicted values
 
 

Start from test set to simulate a predicted dataset

In [173]:
from import_man import *

error_evaluation_columns = ['StoreID', 'Month', 'Region', 'NumberOfSales', '_NumberOfSales']

df = pd.read_csv('./dataset/train.csv')

print("Shape before: " + str(df.shape))

# Let's work on a reduced instance of the test set
df = df.sample(n=500)

print("Shape after: " + str(df.shape))

Shape before: (523021, 36)
Shape after: (500, 36)


In [174]:
from datetime import datetime

# Parse the month and add it as attribute
df['Month'] =  df['Date'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y').month)
df.head()

,StoreID,Date,IsHoliday,IsOpen,HasPromotions,StoreType,AssortmentType,NearestCompetitor,Region,NumberOfCustomers,...,Mean_VisibilityKm,Mean_Wind_SpeedKm_h,Min_Dew_PointC,Min_Humidity,Min_Sea_Level_PressurehPa,Min_TemperatureC,Min_VisibilitykM,Precipitationmm,WindDirDegrees,Month
72560,1104,28/10/2016,0,1,1,Standard Market,With Non-Food Department,19106,5,222,...,12.0,5,6,77,1013,6,5.0,0.25,-1,10
471245,1676,02/08/2016,0,1,1,Shopping Center,General,224,3,845,...,11.0,13,12,43,1012,17,8.0,2.03,279,8
233259,1332,04/11/2016,1,0,0,Hyper Market,General,2065,9,0,...,12.0,19,6,66,1008,8,9.0,0.00,169,11
481868,1690,23/09/2017,0,1,0,Hyper Market,General,2416,9,331,...,11.0,6,13,55,1012,14,8.0,0.00,166,9
283866,1405,01/10/2016,0,1,0,Super Market,With Non-Food Department,4869,0,221,...,10.0,10,2,42,1015,2,0.0,0.00,141,10


In [175]:
# Create fake predicted sales 
df['_NumberOfSales'] = df.NumberOfSales.apply(lambda x: df['NumberOfSales'].sample().values[0])

# Remove useless columns
df =  df[error_evaluation_columns]

df.head(20)

,StoreID,Month,Region,NumberOfSales,_NumberOfSales
72560,1104,10,5,5229,4865
471245,1676,8,3,10191,11642
233259,1332,11,9,0,10717
481868,1690,9,9,4286,1809
283866,1405,10,0,3107,7335
464934,1666,10,10,4195,3619
346670,1495,8,9,3586,996
375480,1537,9,9,4106,4036
136648,1196,3,10,2697,2289
414236,1593,2,9,8538,5619


df.groupby('Region').sum()

abs(df['NumberOfSales'] - df['_NumberOfSales'])


In [177]:
df_sales_sums_by_store_region = df.groupby(['StoreID', 'Month']).sum()

df_sales_sums_by_store_region['abs_a_minus_p'] = abs(df_sales_sums_by_store_region['NumberOfSales'] - df_sales_sums_by_store_region['_NumberOfSales'])
print(df_sales_sums_by_store_region.head(10))


               Region  NumberOfSales  _NumberOfSales  abs_a_minus_p
StoreID Month                                                      
1000    1           7           8902            2697           6205
        9           7           7373           10191           2818
        11          7           9119            4102           5017
1005    9           9           7536            2828           4708
1008    12          9          12140               0          12140
1009    1           0           6477            4207           2270
        4           0           1921            2741            820
1011    4           9           1695           13840          12145
        8          18           2277            2645            368
1013    8           3           5282            4036           1246


In [182]:
N_regions = len(df.Region.unique())